<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09_Chavrusa_04/Get_started_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

# 새 섹션

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Getting started with Gemini models

The new **[Google Gen AI SDK](https://github.com/googleapis/python-genai)** provides a unified interface to [Gemini models](https://ai.google.dev/gemini-api/docs/models) through both the [Gemini Developer API](https://ai.google.dev/gemini-api/docs) and the Gemini API on [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview). With a few exceptions, code that runs on one platform will run on both. This notebook uses the Developer API.

This notebook will walk you through:

* [Installing and setting-up](Get_started.ipynb#scrollTo=Mfk6YY3G5kqp) the Google GenAI SDK
* [Text](Get_started.ipynb#scrollTo=6TYNPrNvQ8ue) and [multimodal](#scrollTo=yww-vrxmRiIy) prompting
* Counting [tokens](Get_started.ipynb#scrollTo=_9B8pb7tv_Cx)
* Setting system instructions
* Configuring [safety filters](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* Initiating a [multi-turn chat](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* [Controlling generated output](Get_started.ipynb#scrollTo=nyZMoM6tgnTA)
* Using [function calling](Get_started.ipynb#scrollTo=Rl-y9SZywD0s)
* Generating a [content stream](Get_started.ipynb#scrollTo=uQfLCxfQtPTg) and sending [asynchronous](Get_started.ipynb#scrollTo=plCtEIaHuv96) requests
* Using [file uploads](Get_started.ipynb#scrollTo=enBhuaIk3KYa)
* Using [context caching](Get_started.ipynb#scrollTo=oTgeR3_9wN5J)
* Generating [text embeddings](Get_started.ipynb#scrollTo=sXNCRn8Wx71d)

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks).

## Setup

### Install SDK

Install the SDK from [PyPI](https://github.com/googleapis/python-genai).

In [1]:
%pip install -U -q 'google-genai>=1.0.0'

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GEMINI_API_KEY`. If you don't already have an API key or you aren't sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [2]:
from google.colab import userdata

GEMINI_API_KEY = userdata.get('gemini-key')

### Initialize SDK client

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [3]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GEMINI_API_KEY)

### Choose a model

Select the model you want to use in this guide. You can either select one from the list or enter a model name manually. Keep in mind that some models, such as the 2.5 ones are thinking models and thus take slightly more time to respond. For more details, you can see [thinking notebook](./Get_started_thinking.ipynb) to learn how to switch the thinking off.

For a full overview of all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini).

In [4]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash-lite-preview-09-2025", "gemini-2.5-flash", "gemini-2.5-flash-preview-09-2025", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Send text prompts

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to `generate_content` and use the `.text` property to get the text content of the response. Note that the `.text` field will work when there's only one part in the output.

In [5]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What's the largest planet in our solar system?"
)

Markdown(response.text)

The largest planet in our solar system is **Jupiter**.

### Upload a PDF file

This PDF page is an article titled [Smoothly editing material properties of objects](https://research.google/blog/smoothly-editing-material-properties-of-objects-with-text-to-image-models-and-synthetic-data/) with text-to-image models and synthetic data available on the Google Research Blog.


In [6]:
import requests
import pathlib # Import pathlib

PDF = "https://storage.googleapis.com/generativeai-downloads/data/Smoothly%20editing%20material%20properties%20of%20objects%20with%20text-to-image%20models%20and%20synthetic%20data.pdf"  # @param {type: "string"}
pdf_bytes = requests.get(PDF).content

pdf_path = pathlib.Path('article.pdf')
pdf_path.write_bytes(pdf_bytes)

6695391

In [9]:
import requests
import pathlib
import time
from IPython.display import Markdown

# --- 2. 파일 업로드 (기존 코드) ---
print("파일 업로드 시작...")
## 기술금융 pdf 검색하여 다운로드
# file_upload = client.files.upload(file=pdf_path)
# file_upload = client.files.upload(file="/content/article.pdf") # Added file= and corrected path
file_upload = client.files.upload(file="/content/184_5.pdf") # Added file= and corrected path

print(f"파일 업로드 완료. 이름: {file_upload.name}, 상태: {file_upload.state}")


# --- 3. (추가된 해결 로직) 파일 상태 확인 및 대기 함수 ---
def wait_for_file_active(client, file_name, timeout=300, interval=5):
    """
    파일의 상태가 'ACTIVE'가 될 때까지 기다립니다.
    """
    start_time = time.time()
    print(f"파일 '{file_name}'의 처리 완료를 기다리는 중...")

    while time.time() - start_time < timeout:
        # 파일 정보를 다시 가져옵니다.
        file = client.files.get(name=file_name)

        if file.state.name == 'ACTIVE':
            print("✅ 파일 처리 완료! 상태: ACTIVE")
            return file
        elif file.state.name == 'FAILED':
            raise Exception(f"❌ 파일 처리 실패! 상태: FAILED. 파일을 삭제하고 다시 업로드해 보세요.")

        # ACTIVE가 아니면 잠시 기다립니다.
        print(f"현재 상태: {file.state.name}. {interval}초 후 재확인...")
        time.sleep(interval)

    raise TimeoutError(f"⏰ 파일 처리 시간 초과 ({timeout}초).")

# --- 4. 파일 상태가 ACTIVE가 될 때까지 대기 (해결 로직 적용) ---
try:
    active_file = wait_for_file_active(client, file_upload.name)
except (TimeoutError, Exception) as e:
    print(f"오류 발생: {e}")
    # 파일 처리가 실패하거나 시간 초과된 경우, 아래 generate_content를 실행하지 않고 종료
    # (선택적) 에러 발생 시 파일 삭제 로직 추가
    # client.files.delete(name=file_upload.name)
    exit()


# --- 5. 모델 호출 (수정된 코드: ACTIVE 상태의 파일을 사용) ---
print("\n모델에게 요약 요청...")
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        active_file, # ACTIVE 상태가 확인된 파일 사용
        "Can you summarize this file as a bulleted list?",
    ]
)

Markdown(response.text)

파일 업로드 시작...
파일 업로드 완료. 이름: files/cduyy01r2ncp, 상태: FileState.ACTIVE
파일 'files/cduyy01r2ncp'의 처리 완료를 기다리는 중...
✅ 파일 처리 완료! 상태: ACTIVE

모델에게 요약 요청...


Here's a summary of the provided document on technology finance as a bulleted list:

*   **Necessity of Technology Finance:**
    *   Essential for a knowledge-based economy, as traditional growth models reliant on imitation and input expansion are reaching their limits.
    *   Addresses market failures in financing R&D, startups, and commercialization, which suffer from high uncertainty, information asymmetry (adverse selection, moral hazard), and external economies.
    *   Despite government intervention, a shift towards market-friendly approaches is needed, especially given the strengthening of risk management under Basel II.

*   **Structure of Korea's Technology Finance Market (2010 data):**
    *   Total size: Approx. 22.8 trillion KRW, accounting for 5.2% of SME loans.
    *   Dominant forms: Technology Evaluation Guarantee (KODIT) at 76%, policy loans at 10%, and venture capital investments at 14%.
    *   Indirect finance (guarantees and policy loans) constitutes 86% of the market.
    *   Government (policy finance) overwhelmingly dominates the market with a 92:8 ratio against private capital.
    *   Contrasts with the US, where venture capital plays a leading role.

*   **Current Status and Problems of Specific Mechanisms:**
    *   **Technology Collateral System (1997-2001):**
        *   An early attempt to use intangible assets (IP) as collateral for loans.
        *   Limited success due to an underdeveloped technology transaction market (difficulty in collateral recovery) and the government's high loss coverage (90%), which diluted its true collateral function.
    *   **KODIT's Technology Evaluation Guarantee:**
        *   The most prominent technology finance product in Korea, combining technology evaluation with guarantees.
        *   While it has shifted to technology-based evaluations, it remains heavily reliant on government funding (90-70% guarantee ratio), making it less of a market-driven product.
        *   KODIT's direct investments may "crowd out" private venture capital.
    *   **Bank's Technology Evaluation Certificate-backed Credit Loans:**
        *   Conceptually market-friendly, as banks extend credit based on technology evaluation certificates (from KODIT, KISTI, etc.) and bear 100% of the credit risk.
        *   Actual adoption is low (significantly decreased after 2006) because banks are reluctant to take on full risk without robust hedging or mitigation.
    *   **Venture Capital (VC):**
        *   Ideal for high-risk, high-return tech finance, but Korea's VC market is stagnant, with public "mother funds" increasingly playing a dominant role.
        *   Private VCs tend to avoid early-stage companies (under 3 years) due to higher risk and institutional investors' preference for stable returns from later-stage companies.
        *   Public mother funds (K-VIC) still have insufficient early-stage investment (e.g., 30% of total investment), not fully fulfilling their market-failure correction role.

*   **Strategies for Revitalizing Technology Finance (Market-Friendly Approaches):**
    *   **Activating Bank's Technology Evaluation Certificate-backed Credit Loans:**
        *   Introduce risk-sharing mechanisms where government/policy funds bear a portion of the credit risk (e.g., 50%+).
        *   Develop credit derivatives (Synthetic CDOs, CDS) to allow banks to transfer/distribute credit risk to capital markets (citing Germany's KfW PROMISE program as an example).
        *   Provide incentives to banks, such as favorable treatment in the Bank of Korea's aggregate credit ceiling loan system.
    *   **Strengthening VC Investment:**
        *   Increase participation from institutional investors (pension funds, corporate VCs) by relaxing regulations on their contributions to VC funds.
        *   Promote Corporate Venture Capital (CVC), potentially through matching funds for new growth sectors.
        *   Introduce "Venture Lease" (equipment financing for startups) to reduce early-stage capital burdens.
        *   Explore credit derivatives (CDS) for VC investments to hedge risk, requiring a public entity to aggregate portfolios for scale.
    *   **Introducing Technology Value Insurance:**
        *   A new mechanism where technology is valued, insured, and financial institutions lend based on the insurance certificate; in case of default, the insurer pays.
        *   Key benefit: Allows the company to retain IP even if it defaults, facilitating rehabilitation.
        *   Requires a policy insurance institution (due to initial high risk), clear risk-sharing, and mechanisms to prevent moral hazard and double-collateralization.
    *   **Market Revitalization through Increased Demand for Technology Evaluation:**
        *   The current technology evaluation system is largely supply-driven and public-sector-dominated.
        *   Need to shift focus to expand private-sector demand for evaluations (e.g., for investment, M&A, IP transactions).
        *   Consolidate fragmented government evaluation bodies into a centralized "Technology Evaluation Institute" (거점기관 - Hub institution) to enhance efficiency, standardization, and trustworthiness.

*   **Conclusion:**
    *   Korea's technology finance market, historically government-led, needs to evolve towards market-friendly models.
    *   Key to activation is effective risk-sharing among all market participants.
    *   Over-reliance on KODIT guarantees should gradually decrease, with increased emphasis on venture capital and diverse financial products.
    *   The aim is to create a virtuous cycle where diversified products and appropriate risk-sharing lead to increased demand for technology evaluation, enhancing market reliability and fostering innovation.